# Unsupervised Domain Adaptation Project


## Part-1: Data download
Load data to project from Google Drive. Copy a subset of classes of images to the path:
- `adaptiope_small/product_images`
- `adaptiope_small/real_life` 

two directories. They represent images from two different domain **product** and **real_life**

In [10]:
from os import makedirs, listdir
from tqdm import tqdm
from google.colab import drive
from os.path import join
from shutil import copytree

drive.mount('/content/gdrive')

!mkdir dataset
!cp "gdrive/My Drive/Colab Notebooks/data/Adaptiope.zip" dataset/
# !ls dataset

!unzip -qq dataset/Adaptiope.zip   # unzip file

!rm -rf adaptiope_small

Mounted at /content/gdrive


In [11]:
!mkdir adaptiope_small
classes = listdir("Adaptiope/product_images")
print(classes)
classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
           "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
           "purse", "stand mixer", "stroller"]
for d, td in zip(["Adaptiope/product_images", "Adaptiope/real_life"], ["adaptiope_small/product_images", "adaptiope_small/real_life"]):
  makedirs(td)
  for c in tqdm(classes):
    c_path = join(d, c)
    c_target = join(td, c)
    copytree(c_path, c_target)

['hat', 'pikachu', 'golf club', 'vr goggles', 'pen', 'hair dryer', 'laptop', 'backpack', 'microwave', 'puncher', 'printer', 'stethoscope', 'wheelchair', 'keyboard', 'bicycle', 'roller skates', 'hot glue gun', 'ice cube tray', 'mixing console', 'lawn mower', 'vacuum cleaner', 'helicopter', 'screwdriver', 'chainsaw', 'hourglass', 'coat hanger', 'stapler', 'purse', 'spatula', 'boxing gloves', 'rubber boat', 'computer mouse', 'tyrannosaurus', 'knife', 'glasses', 'smoking pipe', 'comb', 'hand mixer', 'speakers', 'syringe', 'electric shaver', 'watering can', 'toothbrush', 'tank', 'wristwatch', 'stand mixer', 'skeleton', 'file cabinet', 'toilet brush', 'cordless fixed phone', 'webcam', 'razor', 'ladder', 'motorbike helmet', 'bookcase', 'dart', 'handgun', 'stroller', 'nail clipper', 'ice skates', 'bottle', 'sword', 'office chair', 'fan', 'smartphone', 'grill', 'game controller', 'calculator', 'fire extinguisher', 'brachiosaurus', 'hoverboard', 'binoculars', 'power drill', 'sewing machine', 'pi

100%|██████████| 20/20 [00:05<00:00,  3.97it/s]


## Part-2: Image Classification Neural Network

 

### Part-2.0: Data Loading

First we load the data and preprocessing them

In [12]:
product_path = 'adaptiope_small/product_images'
real_life_path = 'adaptiope_small/real_life'

In [13]:
!pwd
!ls

/content
Adaptiope  adaptiope_small  dataset  gdrive  sample_data


In [14]:
from PIL import Image
from os.path import join

img = Image.open(join(product_path, 'backpack', 'backpack_003.jpg'))
print('Image size: ', img.size)
#img

Image size:  (679, 679)


import libraries

In [15]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16, resnet18, resnet34
from torch.utils.data import DataLoader, random_split

configuration constants

In [ ]:
img_size = 256
# mean, std used by pre-trained models from PyTorch
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
batch_size = 100
learning_rate = 0.001
num_epochs = 15

Configue GPU

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [18]:
from torchvision.transforms.transforms import ToTensor

def get_dataset(root_path):
  '''
    Get dataset from specific data path

    # parameters:
        root_path: path to image folder

    # return: train_loader, test_loader
  '''
  # Construct image transform
  image_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
  ])

  # Load data from filesystem
  image_dataset = ImageFolder(root_path, transform=image_transform)

  return image_dataset

def get_dataloader(dataset, batch_size, shuffle_train=True, shuffle_test=False):
  '''
    Get DataLoader from specific data path

    # parameters:
        dataset: ImageFolder instance
        batch_size: batch_size for DataLoader
        shuffle_train: whether to shuffle training data
        shuffle_test: whether to shuffle test data
  '''
  # Get train, test number
  num_total = len(dataset)
  num_train = int(num_total * 0.8 + 1)
  num_test  = num_total - num_train

  # random split dataset
  data_train, data_test = random_split(dataset, [num_train, num_test])

  # initialize dataloaders
  loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=shuffle_train)
  loader_test  = DataLoader(data_test, batch_size=batch_size, shuffle=shuffle_test)

  return loader_train, loader_test

### Part-2.1 Pretrain Network

Here we use a pretrain Neural Network to start with, then we fine tune it with the data set we have from **Adaptiope** in one domain, and test it on the target domain. Compare the two result, and set the benchmark for later UDA enriched method. 

In [19]:
pd_dataset = get_dataset(product_path)
len(pd_dataset.classes)

20

### Part-2.2 Define the Deep Residual Network

In [20]:
def initialize_model(num_classes, model_type="ResNet"):
  if model_type.startswith("ResNet"):
    model = resnet18(pretrained=True)
    in_features = model.fc.in_features
    model.fc = torch.nn.Linear(in_features=in_features, out_features=num_classes)
  else:
    model = vgg16(pretrained=True)
    in_features = model.classifier[-1].in_features
    model.classifier[-1] = torch.nn.Linear(in_features=in_features, out_features=num_classes)

  return model

In [21]:
# model = initialize_model(20, "vgg")
# count  = 0
# for name, param in model.named_parameters():
#   # if name.startswith('fc'):
#   # print(name) 
#   count += 1
# print(count)

# print(model.__class__.__name__)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

32


### Part-2.3 Cost function

Divide parameters intro two groups, in which the last fully conneted layer with learning_rate, the other layers with 0.1 * learning_rate.

In [23]:
def get_cost_function():
  return torch.nn.CrossEntropyLoss()

### Part-2.4 Optimizer

In [24]:
def get_optimizer(model, learning_rate, weight_decay, momentum):

  # Get model name
  model_name = model.__class__.__name__

  # define final layer name by different model
  if model_name == "ResNet":
    final_layer_name = "fc"
  elif model_name == "VGG":
    final_layer_name = "classifier.6"
  else:
    raise Exception(f'## GET_OPTIMIZER ## - Undefined Model Type {model_name}')

  pre_trained_weights = []
  final_layer_weights = []

  # get all the parameters required gradient updates
  for name, param in model.named_parameters():
    if param.requires_grad == True:
      if name.startswith(final_layer_name):
        final_layer_weights.append(param)
      else:
        pre_trained_weights.append(param)

  # assign parameters to parameters
  optimizer = torch.optim.SGD([
    {'params': pre_trained_weights},
    {'params': final_layer_weights, 'lr': learning_rate}
  ], lr= learning_rate/10, weight_decay=weight_decay, momentum=momentum)
  
  return optimizer

### Part-2.5 Training and Testing Step

In [35]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
  size = len(dataloader.dataset)

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    # compute prediction and loss
    predicts = model(X)
    loss = loss_fn(predicts, y)

    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [26]:
def test_loop(dataloader, model, loss_fn, device):
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      predicts = model(X)
      test_loss += loss_fn(predicts, y).item()
      correct += (predicts.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### Part-2.6 Training

In [29]:
def training(model, train_dataloader, test_dataloader, device, epochs=10, lr=0.001, wd=0.001, momentum=0.9):
  print(f"Learning_rate {lr}, weight_decay {wd}")
  loss_fn = get_cost_function()
  optimizer = get_optimizer(model, lr, wd, momentum)

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)

  print("Done")

In [36]:
# Get train_dataloader, test_dataloader
dataset = get_dataset(product_path)
train_dataloader, test_dataloader = get_dataloader(dataset, 50)

# Get model
model = initialize_model(len(dataset.classes)).to(device)

# Training
training(model, train_dataloader, test_dataloader, device, num_epochs, learning_rate)

Learning_rate 0.001, weight_decay 0.001
Epoch 1
------------------
loss: 3.121286 [    0/ 1601]
Test Error: 
 Accuracy: 56.9%, Avg loss: 2.153765 

Epoch 2
------------------
loss: 2.186185 [    0/ 1601]
Test Error: 
 Accuracy: 81.0%, Avg loss: 1.369971 

Epoch 3
------------------
loss: 1.203209 [    0/ 1601]
Test Error: 
 Accuracy: 90.2%, Avg loss: 0.923603 

Epoch 4
------------------
loss: 0.651633 [    0/ 1601]
Test Error: 
 Accuracy: 92.7%, Avg loss: 0.684645 

Epoch 5
------------------
loss: 0.763621 [    0/ 1601]
Test Error: 
 Accuracy: 94.2%, Avg loss: 0.551654 

Epoch 6
------------------
loss: 0.426481 [    0/ 1601]
Test Error: 
 Accuracy: 94.7%, Avg loss: 0.466251 

Epoch 7
------------------
loss: 0.396500 [    0/ 1601]
Test Error: 
 Accuracy: 94.2%, Avg loss: 0.409144 

Epoch 8
------------------
loss: 0.446180 [    0/ 1601]


KeyboardInterrupt: ignored

## TODO

### TODO: Dataset unzip Google Drive, Copy to folder

TODO: Batch progress number error

Otherwise Continue UDA

## Part-3: UDA 

Here we use Contrastive Domain Adaptation method proposed [here]().
We train the previous network and run the test on both Source Domain and Target Domain. 

## Part-4: Comparison & Discussion
Here we compare the test result from the direct method and the UDA method. 

## Part-5: Conclusion